<a href="https://colab.research.google.com/github/Aravinda89/multi_label_classification/blob/main/ImageWoof_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai==2.0.19 fastcore==1.3.2

     |████████████████████████████████| 188 kB 6.7 MB/s 
     |████████████████████████████████| 46 kB 2.3 MB/s 
     |████████████████████████████████| 5.9 MB 26.6 MB/s 
     |████████████████████████████████| 748.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.6.0 which is inco

In [2]:
import fastai
print(fastai.__version__)

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

2.0.19
Mounted at /content/gdrive


In [3]:
from fastai.vision.all import *
from fastcore.foundation import L

from fastai.callback.fp16 import to_fp16
from fastai.callback.progress import ProgressCallback
from fastai.callback.schedule import fit_one_cycle

from fastai.data.core import Datasets, show_at
from fastai.data.external import untar_data, URLs
from fastai.data.transforms import IntToFloatTensor, Normalize, ToTensor, IndexSplitter, get_image_files, parent_label, Categorize

from fastai.metrics import accuracy

from fastai.vision.augment import aug_transforms, RandomResizedCrop
from fastai.vision.core import PILImage, imagenet_stats
from fastai.vision.learner import cnn_learner

import random

from sklearn.model_selection import StratifiedKFold

from torchvision.models.resnet import resnet34

In [4]:
path = untar_data(URLs.IMAGEWOOF)

File downloaded is broken. Remove /root/.fastai/archive/imagewoof2.tgz and try again.


In [5]:
path.ls()


(#3) [Path('/root/.fastai/data/imagewoof2/val'),Path('/root/.fastai/data/imagewoof2/noisy_imagewoof.csv'),Path('/root/.fastai/data/imagewoof2/train')]

In [6]:
item_tfms = [ToTensor(), RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))]
batch_tfms = [IntToFloatTensor(), *aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=64

In [7]:
train_imgs = get_image_files(path/'train')
tst_imgs = get_image_files(path/'val')

In [8]:
random.shuffle(train_imgs)

In [9]:
len(train_imgs)

9025

In [10]:
len(tst_imgs)

3929

In [11]:
train_imgs[:5]

(#5) [Path('/root/.fastai/data/imagewoof2/train/n02089973/n02089973_1514.JPEG'),Path('/root/.fastai/data/imagewoof2/train/n02096294/n02096294_11008.JPEG'),Path('/root/.fastai/data/imagewoof2/train/n02089973/n02089973_184.JPEG'),Path('/root/.fastai/data/imagewoof2/train/n02087394/n02087394_10518.JPEG'),Path('/root/.fastai/data/imagewoof2/train/n02086240/n02086240_6833.JPEG')]

In [12]:
start_val = len(train_imgs) - int(len(train_imgs)*.2)
idxs = list(range(start_val, len(train_imgs)))
splitter = IndexSplitter(idxs)
splits = splitter(train_imgs)

In [18]:
splits

((#7220) [0,1,2,3,4,5,6,7,8,9...],
 (#1805) [7220,7221,7222,7223,7224,7225,7226,7227,7228,7229...])